<a href="https://colab.research.google.com/github/TGPZTT/NLP_onlab/blob/main/finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

!pip install accelerate -U


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.9 MB/s eta 0:00:00



**training** Töltsd fel a fájlt (articles_3.txt)


In [3]:
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments

In [4]:
def adathalmaz(utvonal, tokenizer, block_meret = 128):
    adathalmaz = TextDataset(
        tokenizer = tokenizer,
        file_path = utvonal,
        block_size = block_meret,
    )
    return adathalmaz


def adat_collator(tokenizer, mlm = False):
    adat_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=mlm,
    )
    return adat_collator


def train(model_name,
          trening_szoveg,
          output_dir,
          per_device_train_batch_size,
          num_train_epochs,
          save_steps):
  tokenizer = GPT2Tokenizer.from_pretrained(model_name)
  train_dataset = adathalmaz(trening_szoveg, tokenizer)
  data_collator = adat_collator(tokenizer)

  tokenizer.save_pretrained(output_dir)

  model = GPT2LMHeadModel.from_pretrained(model_name)

  model.save_pretrained(output_dir)

  training_args = TrainingArguments(
          output_dir=output_dir,
          overwrite_output_dir=False,
          per_device_train_batch_size=per_device_train_batch_size,
          num_train_epochs=num_train_epochs,
      )

  trainer = Trainer(
          model=model,
          args=training_args,
          data_collator=data_collator,
          train_dataset=train_dataset,
  )

  trainer.train()
  trainer.save_model()

In [30]:
# Ez kb 10 perces művelet.
train(
      'NYTK/PULI-GPT-2',
      "/content/Articles_3.txt",
      '/content/ujmodel',
      10,
      10.0,
      450,
)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Step,Training Loss
500,1.332500


In [8]:
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel, GPT2TokenizerFast, GPT2Tokenizer

In [19]:
model_path = "/content/ujmodel"

def generate_text(input_text, max_length):

    model = GPT2LMHeadModel.from_pretrained(model_path)
    tokenizer = tokenizer = GPT2Tokenizer.from_pretrained(model_path)
    ids = tokenizer.encode(f'{input_text}', return_tensors='pt')
    output = model.generate(
        ids,
        do_sample=True,
        max_new_tokens=max_length,
        top_k=40,
        top_p=0.97,
        pad_token_id=model.config.eos_token_id,
    )
    return(tokenizer.decode(output[0], skip_special_tokens=True))

In [28]:
input_text = "A kedvenc szalámi márkám a Pick"
max_length = 6

riport = generate_text(input_text, max_length)

In [29]:
def anonim(string):
    blacklist = ["Pick", "Nógrádi", "Herz", "Nike", "Stabilo"]
    cserelt_string = string
    for item in blacklist:
        cserelt_string = cserelt_string.replace(item, "*****")
    return cserelt_string

anon_string = anonim(riport)
print("Eredeti string:", riport)
print("Módosított string:", anon_string)

Eredeti string: A kedvenc szalámi márkám a Pick szalámi, ami árban
Módosított string: A kedvenc szalámi márkám a ***** szalámi, ami árban
